DTW
---
It’s a technique used to dynamically compare time series data when the time indices between comparison data points do not sync up perfectly.

In general, DTW is a method that calculates an optimal match between two given sequences (e.g. time series) with certain restriction and rules:

- Every index from the first sequence must be matched with one or more indices from the other sequence, and vice versa
- The first index from the first sequence must be matched with the first index from the other sequence (but it does not have to be its only match)
- The last index from the first sequence must be matched with the last index from the other sequence (but it does not have to be its only match)
- The mapping of the indices from the first sequence to indices from the other sequence must be monotonically increasing, and vice versa.

The optimal match is denoted by the match that satisfies all the restrictions and the rules and that has the minimal cost, where the cost is computed as the sum of absolute differences, for each matched pair of indices, between their values.

---
Pseudo Code

```
int DTWDistance(s: array [1..n], t: array [1..m]) {
    DTW := array [0..n, 0..m]
    
    for i := 1 to n
        for j := 1 to m
            DTW[i, j] := infinity
    DTW[0, 0] := 0
    
    for i := 1 to n
        for j := 1 to m
            cost := d(s[i], t[j])
            DTW[i, j] := cost + minimum(DTW[i-1, j  ],    // insertion
                                        DTW[i  , j-1],    // deletion
                                        DTW[i-1, j-1])    // match
    
    return DTW[n, m]
}
```
where $DTW[i, j]$ is the distance between $s[1:i]$ and $t[1:j]$ with the best alignment.


In [1]:
import numpy as np

In [14]:
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [18]:
a = [1, 2, 3]
b = [2, 2, 2, 3, 4]

dtw(a, b)

array([[ 0., inf, inf, inf, inf, inf],
       [inf,  1.,  2.,  3.,  5.,  8.],
       [inf,  1.,  1.,  1.,  2.,  4.],
       [inf,  2.,  2.,  2.,  1.,  2.]])

Add Window Constraint
---
We sometimes want to add a locality constraint. That is, we require that if $s[i]$ is matched with $t[j]$, then $|i - j|$ is no larger than $w$, a window parameter.

We can easily modify the above algorithm to add a locality constraint (differences marked). However, the above given modification works only if $| n - m |$ is no larger than $w$, i.e. the end point is within the window length from diagonal. In order to make the algorithm work, the window parameter $w$ must be adapted so that $|n-m| \leq w$

---
Pseudo Code
```
int DTWDistance(s: array [1..n], t: array [1..m], w: int) {
    DTW := array [0..n, 0..m]

    w := max(w, abs(n-m)) // adapt window size (*)

    for i := 0 to n
        for j:= 0 to m
            DTW[i, j] := infinity
    DTW[0, 0] := 0
    
    for i := 1 to n
        for j := max(1, i-w) to min(m, i+w)
            DTW[i, j] := 0

    for i := 1 to n
        for j := max(1, i-w) to min(m, i+w)
            cost := d(s[i], t[j])
            DTW[i, j] := cost + minimum(DTW[i-1, j  ],    // insertion
                                        DTW[i  , j-1],    // deletion
                                        DTW[i-1, j-1])    // match

    return DTW[n, m]
}
```

In [37]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    print("inf", dtw_matrix)
    
#     for i in range(1, n+1):
#         for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
#             dtw_matrix[i, j] = 0
#     print("con\n", dtw_matrix)
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [36]:
a = [1, 2, 3, 3, 5]
b = [1, 2, 2, 2, 2, 2, 2, 4]

dtw(a, b, window=2)

inf [[ 0. inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]]
con
 [[ 0. inf inf inf inf inf inf inf inf]
 [inf  0.  0.  0.  0. inf inf inf inf]
 [inf  0.  0.  0.  0.  0. inf inf inf]
 [inf  0.  0.  0.  0.  0.  0. inf inf]
 [inf  0.  0.  0.  0.  0.  0.  0. inf]
 [inf inf  0.  0.  0.  0.  0.  0.  0.]]


array([[ 0., inf, inf, inf, inf, inf, inf, inf, inf],
       [inf,  0.,  1.,  2.,  3., inf, inf, inf, inf],
       [inf,  1.,  0.,  0.,  0.,  0., inf, inf, inf],
       [inf,  3.,  1.,  1.,  1.,  1.,  1., inf, inf],
       [inf,  5.,  2.,  2.,  2.,  2.,  2.,  2., inf],
       [inf, inf,  5.,  5.,  5.,  5.,  5.,  5.,  3.]])

In [38]:
a = [1, 2, 3, 3, 5]
b = [1, 2, 2, 2, 2, 2, 2, 4]

dtw(a, b, window=2)

inf [[ 0. inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf]]


array([[ 0., inf, inf, inf, inf, inf, inf, inf, inf],
       [inf,  0.,  1.,  2.,  3., inf, inf, inf, inf],
       [inf,  1.,  0.,  0.,  0.,  0., inf, inf, inf],
       [inf,  3.,  1.,  1.,  1.,  1.,  1., inf, inf],
       [inf,  5.,  2.,  2.,  2.,  2.,  2.,  2., inf],
       [inf, inf,  5.,  5.,  5.,  5.,  5.,  5.,  3.]])